In [15]:
import pandas as pd
from openpyxl import Workbook
import numpy as np

In [26]:
CSV_Filename = 'name.csv'

In [27]:
def start_excel_workbook_with_sheet_name(sheetName="Summary"):
    wb = Workbook()
    ws = wb.active
    ws.title = sheetName
    row = 0
    return wb

In [28]:
def write_excel_workbook(wb, fileName="junk.xlsx"):
    wb.save(filename = fileName)

In [39]:
def write_to_cell_in_sheet_row_col(ws, row, col, value, format='general'):

    if format == 'number':
        ws.cell(row=row, column=col, value=float(value)).number_format = '0.00'
    elif format == 'scientific':
        ws.cell(row=row, column=col, value=float(value)).number_format = '0.00E+00'
    else:
        ws.cell(row=row, column=col, value=value)

In [40]:
def update_melts_excel_output_workbook(wb, comp_dict, params):
        
    condSh = wb["init_cond"]
    # NEED TO GET OXIDE COMPOSITION OF SYSTEM
    oxides = [i for i in comp_dict.keys()]
    for i in range(0,len(oxides)):
        write_to_cell_in_sheet_row_col(condSh, i+1, 1, oxides[i])
        write_to_cell_in_sheet_row_col(condSh, i+1, 2, comp_dict.get(oxides[i]))

        write_to_cell_in_sheet_row_col(condSh, 1, 4,  "T1 (C)")
        write_to_cell_in_sheet_row_col(condSh, 2, 4,  "T2 (C)")
        write_to_cell_in_sheet_row_col(condSh, 3, 4,  "T step (C)")
        write_to_cell_in_sheet_row_col(condSh, 5, 4, "P1 (MPa)")
        write_to_cell_in_sheet_row_col(condSh, 6, 4, "P2 (MPa)")
        write_to_cell_in_sheet_row_col(condSh, 7, 4, "P step (MPa)")
        write_to_cell_in_sheet_row_col(condSh, 9, 4, "fO2 value")
        write_to_cell_in_sheet_row_col(condSh, 10, 4, "fO2 buffer")

        write_to_cell_in_sheet_row_col(condSh, 1, 5, params[0])
        write_to_cell_in_sheet_row_col(condSh, 2, 5, params[1])
        write_to_cell_in_sheet_row_col(condSh, 3, 5, params[2])
        write_to_cell_in_sheet_row_col(condSh, 5, 5, params[3])
        write_to_cell_in_sheet_row_col(condSh, 6, 5, params[4])
        write_to_cell_in_sheet_row_col(condSh, 7, 5, params[5])
        write_to_cell_in_sheet_row_col(condSh, 9, 5, params[6])
        write_to_cell_in_sheet_row_col(condSh, 10, 5, params[7])

        CalcType = params[9]
        MELTS_Model = params[10]

        if CalcType == 'HP_Sequence':
            write_to_cell_in_sheet_row_col(condSh, 12, 4, "H step (kJ)")
            write_to_cell_in_sheet_row_col(condSh, 12, 5, delta_H)

        if CalcType == "TV_Sequence":
            write_to_cell_in_sheet_row_col(condSh, 12, 4, "V step (cm3)")
            write_to_cell_in_sheet_row_col(condSh, 12, 5, delta_V)

        if CalcType == "SP_Sequence":
            write_to_cell_in_sheet_row_col(condSh, 12, 4, "S step (kJ)")
            write_to_cell_in_sheet_row_col(condSh, 12, 5, delta_S)

        write_to_cell_in_sheet_row_col(condSh, 1, 7, "Model")
        write_to_cell_in_sheet_row_col(condSh, 2, 7, "Calculation")
        write_to_cell_in_sheet_row_col(condSh, 1, 8, "rhyolite-Melts_v"+MELTS_Model[:-2]+".x")
        write_to_cell_in_sheet_row_col(condSh, 2, 8, CalcType)

        if (CalcType == "Draw_Phase_Diagram" and Include_P_Calc == True):
            write_to_cell_in_sheet_row_col(condSh, 2, 8, "QF_P_Calc")


## Loop to create initial condition sheets for all compositions, 

In [42]:
comps = pd.read_csv(CSV_Filename).replace(np.nan, 0, regex=True).set_index(' ', drop = True)
for c in comps:
    comp = comps[c]
    composition = comp[:15]
    composition = pd.to_numeric(composition).to_dict()
    T1 = float(comp['T1'])
    T2 = float(comp['T2'])
    delta_T = float(comp['?T'])
    P1 = float(comp['P1'])
    P2 = float(comp['P2'])
    delta_P = float(comp['?P'])
    const_fO2 = comp['fO2 constraint']
    fO2Path = comp['fO2 buffer']
    fO2_offset = float(comp['fO2 offset'])
    # parameters for modeling to be saved
    params = [T1, T2, delta_T, P1, P2, delta_P, fO2_offset, fO2Path, const_fO2]

    # might be unnecessary
    print('T conditions:', T1, T2, delta_T)
    print('P conditions:', P1, P2, delta_P)
    print('fO2 conditions:', const_fO2, fO2Path, fO2_offset)

    CalcType = comp['Calculation']
    params.append(CalcType)
    MELTS_Model = comp['Model']
    # the below might be unnecessary
    if MELTS_Model == '1.0.x':
        MELTS_Model = '1.0.2'
    elif MELTS_Model == '1.2.x':
        MELTS_Model = '1.2.0'
    elif MELTS_Model == '5.6.x':
        MELTS_Model = '5.6.1'
    elif MELTS_Model == '1.1.x':
        MELTS_Model = '1.1.0'
    params.append(MELTS_Model)
    
    wb = start_excel_workbook_with_sheet_name(sheetName="init_cond")
    update_melts_excel_output_workbook(wb, composition, params)
    write_excel_workbook(wb, fileName="Test.xlsx")

T conditions: 1100.0 700.0 1.0
P conditions: 150.0 50.0 25.0
fO2 conditions: TRUE NNO 0.0
